<a href="https://colab.research.google.com/github/tskmr/MIP/blob/main/Lec11_Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
library(tidyverse)

t = c(
  6700, 6950, 7820, 8790, 9120, 9660, 9820, 11310, 11690, 11850,
  11880, 12140, 12200, 12870, 13150, 13330, 13470, 14040, 14300,
  17520, 17540, 17890, 18450, 18960, 18980, 19410, 20100, 20100,
  20150, 20320, 20900, 22700, 23490, 26510, 27410, 27490, 27890,
  28100)

dj = c(
  1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
  0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0)

rj = 1-dj
n = length(t)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


＃最尤推定

In [3]:
dsev = function(z){
  exp(z-exp(z))
}
psev = function(z){
  1-exp(-exp(z))
}
ft = function(x, par){
  mu = par[1]
  sigma = par[2]
  1/x/sigma * dsev((log(x) - mu)/sigma)
}
Ft = function(x, par){
  mu = par[1]
  sigma = par[2]
  psev((log(x) - mu)/sigma)
}
l_i = function(x, par, n_d, n_r){
  n_d * log(ft(x, par)) + n_r * log(1-Ft(x, par))
}
loglikelihood = function(x, n_d, n_r){
  function(par){
    sum(l_i(x, par, n_d, n_r))
  }
}
l = loglikelihood(t, dj, rj)
res = optim(c(30, 1), l, control= list(fnscale = -1), hessian = TRUE)
res

Hn = res$hessian # ヘッセ行列の推定値
hat.FI = -Hn # 観測フィッシャー情報行列の推定値
hat.aVar = solve(hat.FI) # 逆行列を求めて，プラグイン漸近分散を得る
hat.aVar

hat.mu = res$par[1] # muの最尤推定値
z = qnorm(0.975) # 標準正規分布の上側0.025%点
hat.mu - z * sqrt(hat.aVar[1,1])
hat.mu + z * sqrt(hat.aVar[1,1])

hat.sigma = res$par[2] # muの最尤推定値
z = qnorm(0.975) # 標準正規分布の上側0.025%点
hat.sigma - z * sqrt(hat.aVar[2,2])
hat.sigma + z * sqrt(hat.aVar[2,2])


Warning message in log(ft(x, par)):
“NaNs produced”


-109.86800,81.82946
81.82946,-247.60645


0.012073672,0.003990131
0.003990131,0.005357333


[1] 10.01443

[1] 10.44515

[1] 0.1730241

[1] 0.4599386